In [ ]:
%matplotlib inline
%run intersections.ipynb
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pdb
import pandas
import pickle
import os

#mated and nonmated are ndarrays
def fnmr_vs_fmr(mated, nonmated):
    
    m0_mated = mated.min()
    m1_nonmated = nonmated.max()
    num_nonmated = len(nonmated)
    num_mated = len(mated)
    fnmr = []
    fmr = []
    
    if m0_mated > m1_nonmated: #never cross
        #print('m0_mated > m1_nonmated')
        return [], [], np.nan, np.nan
    else:
        #step = (m1_nonmated-m0_mated)/100
        x = np.arange(m0_mated, m1_nonmated, 0.0001)

    for i in x:
        fr = 0
        fa = 0
        for m in np.arange(0, num_mated-1):
            if mated[m] < i:
                fr = fr+1
        for nm in np.arange(0, num_nonmated-1):
            if nonmated[nm] >= i:
                fa = fa+1
        fnmr = np.append(fnmr, 100*fr/num_mated)
        fmr = np.append(fmr, 100*fa/num_nonmated)
    
    if fnmr.size == 0 or fmr.size == 0:
        #print('empty fnmr or fmr')
        return [], [], np.nan, np.nan
        
    #####eer calculation
    #
    #Find where fnmr and fmr cross.
    #Find where the curves have the least difference and in which x point that happens
    subs = fmr-fnmr
    
    if sum(subs) == 0 or sum(subs) == 0:
        #print('fnmr and fmr are the same value all the way')
        return fnmr, fmr, np.nan, np.nan

    min_diff1 = subs[subs > 0]
    min_diff2 = subs[subs < 0]

    if min_diff1.size > 0:
        min_diff1 = min_diff1.min()
        if min_diff1.size > 1:
            #print('min_diff1 is found in more than one point')
            min_diff1 = min_diff1[0]
    
    if min_diff2.size > 0:
        min_diff2 = min_diff2.max()
        if min_diff2.size > 1:
            #print('min_diff2 is found in more than one point')
            min_diff2 = min_diff2[min_diff2.size - 1]
                              
    if min_diff2.size == 0 or min_diff1.size == 0:
        #print('fnmr and fmr do not cross')
        return [], [], np.nan, np.nan
    
    idx1 = np.where(subs == min_diff1)[0]
    idx2 = np.where(subs == min_diff2)[0]
    
    if idx1.size > 1:
        idx1 = idx1[0]
    if idx2.size > 1:
        idx2 = idx2[idx2.size - 1]
    
    A = np.array([[x[idx1], x[idx2]], [fnmr[idx1], fnmr[idx2]]])
    B = np.array([[x[idx1], x[idx2]], [fmr[idx1], fmr[idx2]]])
    
    x_eer, eer = intersections(A, B)

    #Plot FNMR vs FMR
    #if fnmr.size > 0 and fmr.size > 0:
    #    fig = plt.plot(x, fnmr, 'b', x, fmr, 'r')
    #    plt.scatter(x_eer, eer, color='black' )
    #    plt.ylabel('%')
    #    plt.xlabel('Threshold')
    #    plt.show()
    #print('EER = ', eer)
    return fnmr, fmr, eer, x_eer
